In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import zipfile
import sys
import time

# Any results you write to the current directory are saved as output.

['sample_submission_stage_1.csv', 'sample_submission_stage_2.csv', 'test_stage_2.tsv', 'test_stage_1.tsv']


**Refer this to get started with pre trained BERT**

https://github.com/google-research/bert#pre-trained-models


In [2]:
!ls

__notebook_source__.ipynb


In [3]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip", "r") as bert_zip_ref:
    bert_zip_ref.extractall()
!ls 'uncased_L-12_H-768_A-12'

--2019-05-20 17:18:00--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.184.128, 2a00:1450:400c:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.184.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   130MB/s    in 3.0s    

2019-05-20 17:18:03 (130 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


Download requirements to use pretrained BERT

In [4]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/extract_features.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

--2019-05-20 17:19:40--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.005s  

2019-05-20 17:19:40 (6.88 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-05-20 17:19:41--  https://raw.githubusercontent.com/google-research/bert/master/extract_features.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13898 (14K) [text/plain]
Saving to: ‘extrac

In [5]:
import tokenization
import extract_features
import modeling
import tensorflow as tf

Download data from GAP repo. The gap-validation.tsv file with 454 rows, and gap-test.tsv with 2000 rows, will be used for training. The gap-development.tsv data contains the same 2000 rows as test_stage_1.tsv data. So we'll make predictions on it.

In [6]:
!wget https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv
!wget https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv
!wget https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv
!ls

--2019-05-20 17:19:55--  https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1075889 (1.0M) [text/plain]
Saving to: ‘gap-test.tsv’

gap-test.tsv        100%[===================>]   1.03M  --.-KB/s    in 0.03s   

2019-05-20 17:19:55 (29.4 MB/s) - ‘gap-test.tsv’ saved [1075889/1075889]

--2019-05-20 17:19:56--  https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

In [7]:
def compute_offset_no_space(text, offset):
    count = 0
    for position in range(offset):
        if text[position] != " ":
            count += 1
    return count

def count_chars_no_special(text):
    count = 0
    special_chars = ["#"]
    for pos in range(len(text)):
        if text[pos] not in special_chars:
            count += 1
    return count

def count_length_no_special(text):
    count = 0
    special_chars =["#", " "]
    for pos in range(len(text)):
        if text[pos] not in special_chars:
            count += 1
    return count

Obtain Contextual Embeddings for the target words. Run the below method 3 times for each of the dataset files

In [10]:
def get_contextual_embeddings(data):
    '''
    Runs a forward propagation of BERT on input text, extracting contextual word embeddings
    Input: data, a pandas DataFrame containing the information in one of the GAP files

    Output: emb, a pandas DataFrame containing contextual embeddings for the words A, B and Pronoun. 
            Each embedding is a numpy array of shape (768)
    columns: "emb_A": the embedding for word A
             "emb_B": the embedding for word B
             "emb_P": the embedding for the pronoun
             "label": the answer to the coreference problem: "A", "B" or "NEITHER"
    '''
    # take the text part and pass it to bert
    text = data["Text"]
    text.to_csv("input.txt", index=False, header=False)
    
    # extract_features.py runs forward propagation throught bert
    
    os.system("python3 extract_features.py \
              --input_file=input.txt \
              --output_file=output.jsonl \
              --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
              --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
              --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
              --layers=-1 \
              --max_seq_length=256 \
              --batch_size=8")
    bert_outs = pd.read_json("output.jsonl", lines=True)
    
    # delete both files input.txt and output.jsonl
    os.system("rm input.txt")
    os.system("rm output.jsonl")
    
    index = data.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index=index, columns=columns)
    emb.index.name = "ID"
    
    for i in range(len(data)):
        P = data.loc[i, "Pronoun"].lower()
        A = data.loc[i, "A"].lower()
        B = data.loc[i, "B"].lower()
        
        # for each sentence count the offset not counting spaces
        p_offset = compute_offset_no_space(data.loc[i, "Text"], data.loc[i, "Pronoun-offset"])
        A_offset = compute_offset_no_space(data.loc[i, "Text"], data.loc[i, "A-offset"])
        B_offset = compute_offset_no_space(data.loc[i, "Text"], data.loc[i, "B-offset"])
        
        # compute length
        A_length = count_length_no_special(A)
        B_length = count_length_no_special(B)
        
        emb_A = np.zeros(768)
        emb_B = np.zeros(768)
        emb_P = np.zeros(768)
        
        # init counts
        count_chars = 0
        count_A, count_B, count_C = 0, 0, 0
        
        # get bert embeddings for current line
        bert_feat = pd.DataFrame(bert_outs.loc[i, "features"])
        
        # Iterate over the BERT tokens for the current line; we skip over the first 2 tokens, which don't correspond to words
        # See if the character count until the current token matches the offset of any of the 3 target words
        for j in range(2, len(bert_feat)):
            token = bert_feat.loc[j, "token"]
            if count_chars == p_offset:
                emb_P += np.array(bert_feat.loc[j, "layers"][0]['values'])
                count_chars += 1
            
            if count_chars in range(A_offset, A_offset + A_length):
                emb_A += np.array(bert_feat.loc[j, "layers"][0]['values'])
                count_A += 1
            
            if count_chars in range(B_offset, B_offset + B_length):
                emb_B += np.array(bert_feat.loc[j, "layers"][0]['values'])
                count_B += 1
            count_chars += count_length_no_special(token)
        
        # Taking the average between tokens in the span of A or B, so divide the current value by the count	
        emb_A /= count_A
        emb_B /= count_B
        
        label = "None"
        if data.loc[i, "A-coref"] == True:
            label = "A"
        if data.loc[i, "B-coref"] == True:
            label = "B"
            
        emb.iloc[i] = [emb_A, emb_B, emb_P, label]
    return emb
        

Read the three dataset files pass them to the above method and get the bert embeddings for pronoun, A and B

In [11]:
# read file
validation_data = pd.read_csv("gap-validation.tsv", sep='\t')
test_data = pd.read_csv("gap-test.tsv", sep='\t')
development_data = pd.read_csv("gap-development.tsv", sep='\t')

# get contextual embeddings from BERT pretrained
validation_embeddings = get_contextual_embeddings(validation_data)
test_embeddings = get_contextual_embeddings(test_data)
development_embeddings = get_contextual_embeddings(development_data)

# write embeddings to json file
validation_embeddings.to_json("validation_emb.json")
test_embeddings.to_json("test_emb.json")
development_embeddings.to_json("dev_emb.json")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:74: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in true_divide


In [42]:
from keras import layers, models, optimizers
from keras import callbacks as k_callback
from keras import backend as K
from keras import constraints, initializers, regularizers
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import log_loss
import time

# hyperparameters
num_units = 64
learning_rate = 0.001
dropout_rate = 0.5
patience = 100
nfold = 5
epochs = 1200
lmbd = 0.1
batch_size = 32

In [37]:
def build_model(input_shape):
    input_ = layers.Input(shape=input_shape)
    X = layers.Dense(units=num_units, name='dense_1')(input_)
    X = layers.BatchNormalization(name='bn_1')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed=7)(X)
    
    X = layers.Dense(units=num_units, name='dense_2')(X)
    X = layers.BatchNormalization(name='bn_2')(X)
    X = layers.Activation('relu')(X)
    X = layers.Dropout(dropout_rate, seed=7)(X)
    
    X = layers.Dense(3, kernel_regularizer=regularizers.l2(lmbd), name='output_layer')(X)
    X = layers.Activation('softmax')(X)
    
    model = models.Model(inputs=input_, outputs=X, name='classifier')
    return model

In [19]:
def parse_json(embeddings):
    '''
    Parses the embeddigns given by BERT, and suitably formats them for the MLP model

    Input: embeddings, a DataFrame containing contextual embeddings from BERT, 
           as well as the labels for the classification problem
    columns: "emb_A": contextual embedding for the word A
             "emb_B": contextual embedding for the word B
             "emb_P": contextual embedding for the pronoun
         "label": the answer to the coreference problem: "A", "B" or "None"

    Output: X, a numpy array, for each line in the GAP file,
            the concatenation of the embeddings of the target words
            Y, a numpy array , for each line in the GAP file,
            the one-hot encoded label
            '''
    embeddings.sort_index(inplace=True)
    
    # initialise X and Y
    X = np.zeros((len(embeddings), 3*768))
    Y = np.zeros((len(embeddings), 3))
    
    for i in range(len(embeddings)):
        A = np.array(embeddings.loc[i, "emb_A"])
        B = np.array(embeddings.loc[i, "emb_B"])
        C = np.array(embeddings.loc[i, "emb_P"])
        X[i] = np.concatenate((A, B, C))
    
    for i in range(len(embeddings)):
        label = embeddings.loc[i, "label"]
        if label == "A":
            Y[i, 0] = 1
        elif label == "B":
            Y[i, 1] = 1
        else:
            Y[i, 2] = 1
    return X, Y

Parse embeddings stored in json file using the abpve method to get in the format as required by the MLP model for all the three datasets

In [49]:
valid = pd.read_json("validation_emb.json")
X_valid, Y_valid = parse_json(valid)

dev = pd.read_json("dev_emb.json")
X_dev, Y_dev = parse_json(dev)

test = pd.read_json("test_emb.json")
X_test, Y_test = parse_json(test)

In [51]:
# Remove Nan, the offset of a target word is greater than the max_seq_length of BERT. are all Nans
dev_nan = [row for row in range(len(X_dev)) if np.sum(np.isnan(X_dev[row]))]
X_dev[dev_nan] = np.zeros(3*768)

dev_val = [row for row in range(len(X_valid)) if np.sum(np.isnan(X_valid[row]))]
X_valid = np.delete(X_valid, dev_val, axis=0)
Y_valid = np.delete(Y_valid, dev_val, axis=0)

dev_test = [row for row in range(len(X_test)) if np.sum(np.isnan(X_test[row]))]
X_test = np.delete(X_test, dev_test, axis=0)
Y_test = np.delete(Y_test, dev_test, axis=0)

Concatenate test and validation datasets for training with K-fold cross validation. Using development dataset for testing.

In [52]:
X_train = np.concatenate((X_test, X_valid), axis=0)
Y_train = np.concatenate((Y_test, Y_valid), axis=0)

predictions = np.zeros((len(X_dev), 3))

In [53]:
# KFold cross validation
kfold = KFold(n_splits=nfold, shuffle=True, random_state=3)
scores = []
for fold, (train_index, val_index) in enumerate(kfold.split(X_train)):
    # print time of fold start
    print("fold: {0}, time: {1}".format(fold, time.ctime()))
    
    # split dataset
    x_train, x_val = X_train[train_index], X_train[val_index]
    y_train, y_val = Y_train[train_index], Y_train[val_index]
    
    # build mlp model
    model = build_model([X_train.shape[1]])
    model.compile(optimizer=optimizers.Adam(lr=learning_rate), loss='categorical_crossentropy')
    callback = [k_callback.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)]
    model.fit(x=x_train,
             y=y_train,
             batch_size=batch_size,
             epochs=epochs,
             callbacks=callback,
             validation_data=(x_val, y_val),
             verbose=0)
    preds_validation = model.predict(x_val, verbose=0)
    predictions += model.predict(X_dev, verbose=0)
    
    scores.append(log_loss(y_val, preds_validation))
predictions /= nfold
print("Cross valid mean score: {0:.4f}, std: {1:.4f}".format(np.mean(scores), np.std(scores)))
print(scores)
print("Test Scores: {0:.5f}".format(log_loss(Y_dev, predictions)))
    

fold: 0, time: Mon May 20 20:20:24 2019
fold: 1, time: Mon May 20 20:21:35 2019
fold: 2, time: Mon May 20 20:22:47 2019
fold: 3, time: Mon May 20 20:23:59 2019
fold: 4, time: Mon May 20 20:25:14 2019
Cross valid mean score: 0.5704, std: 0.0203
[0.5879626486705551, 0.5325747891016259, 0.5874970128186912, 0.5757299818127055, 0.5680406794058306]
Test Scores: 0.53170


In [54]:
submission = pd.read_csv("../input/sample_submission_stage_1.csv", index_col = "ID")
submission["A"] = predictions[:, 0]
submission["B"] = predictions[:, 1]
submission["NEITHER"] = predictions[:, 2]
submission.to_csv("submission_bert.csv")